The point of this exercise is implementing from scratch, which means only using Numpy, not even using a single feature of Scikit-learn, expect for loading the iris dataset.

In [2]:
import numpy as np

# Load the data

In [1]:
from sklearn.datasets import load_iris

iris = load_iris(as_frame=True)

X = iris.data.values
y = iris.target

We need to add $x_0=1$ in the front of every instance

# Spilt the data

Of course, it would be very easy to use train_test_spilt in Scikit-learn, but we're not here for the easy stuff.

In [ ]:
test_ratio = 0.2
validation_ratio = 0.2

np.random.seed(42)

np.random.permutation()